# 이 노트북에서의 특징 변환

이 노트북에서는 원시 텍스트를 토큰화하여 허깅페이스 훈련 스크립트에서 사용할 수 있는 입력으로 변환합니다. 이 과정에서 토큰화와 프롬프트 생성 단계를 훈련 과정과 분리하는 것이 중요합니다. 이를 통해 각 단계에 가장 적합한 컴퓨팅 자원을 사용할 수 있습니다. 예를 들어 저렴한 CPU는 데이터 준비 단계에 가장 적합하고 고성능 GPU는 모델 훈련에 가장 효율적입니다.

## 준비사항
아래 실습은 AWS ml.m5.2xlarge 인스턴스에서 수행했습니다.

## 커널 및 필요한 종속성 설정하기

In [ ]:
%pip install --disable-pip-version-check \
    torch==2.0.1 \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict
import os
import time

## 기본 데이터 세트가 다운로드되었는지 확인하기

In [ ]:
from datasets import concatenate_datasets
dataset = load_dataset("knkarthick/dialogsum")
dataset = concatenate_datasets([dataset['train'], dataset['test'], dataset['validation']])
!mkdir data-summarization
dataset = dataset.train_test_split(0.5, seed=1234)
dataset['test'].to_csv('./data-summarization/dialogsum-1.csv', index=False)
dataset['train'].to_csv('./data-summarization/dialogsum-2.csv', index=False)

## 토크나이저 및 허깅페이스 데이터 세트 불러오기

In [ ]:
model_checkpoint='google/flan-t5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
dataset = load_dataset('./data-summarization/')
dataset

## 예제 프롬프트 탐색하기

In [ ]:
idx = 0
diag = dataset['train'][idx]['dialogue']
baseline_human_summary = dataset['train'][idx]['summary']

prompt = f'Summarize the following conversation.\n\n{diag}\n\nSummary:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

print(f'프롬프트:\n--------------------------\n{prompt}\n--------------------------')
print(f'기본 인간 요약 : {baseline_human_summary}')

## 데이터 세트 토큰화하기

In [ ]:
def tokenize_function(example):
    prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    inp = [prompt + i + end_prompt for i in example["dialogue"]]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

## 전처리 과정을 반복 가능한 함수로 만들기

In [ ]:
def tokenize_function(example):
    prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    inp = [prompt + i + end_prompt for i in example["dialogue"]]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

def transform_dataset(input_data,
                      output_data,
                      huggingface_model_name,
                      train_split_percentage,
                      test_split_percentage,
                      validation_split_percentage,
                      ):

    # 원본 데이터 세트 불러오기
    dataset = load_dataset(input_data)
    print(f'데이터 세트를 다음 경로에서 불러왔습니다: {input_data}\n{dataset}')
    
    # 모델의 토크나이저 불러오기
    print(f'모델 {huggingface_model_name}의 토크나이저를 불러오는 중...')
    tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name)
    
    # 훈련, 테스트, 검증 데이터 분할
    train_testvalid = dataset['train'].train_test_split(1 - train_split_percentage, seed=1234)
    test_valid = train_testvalid['test'].train_test_split(test_split_percentage / (validation_split_percentage + test_split_percentage), seed=1234)
    train_test_valid_dataset = DatasetDict(
        {
            'train': train_testvalid['train'],
            'test': test_valid['test'],
            'validation': test_valid['train']
        }
    )
    print(f'데이터 세트 분할 후:\n{train_test_valid_dataset}')
    
    # 데이터 세트 토큰화
    print(f'데이터 세트 토큰화하는 중...')
    tokenized_datasets = train_test_valid_dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])
    print(f'토큰화 완료!')
    
    # 저장할 디렉토리 생성
    os.makedirs(f'{output_data}/train/', exist_ok=True)
    os.makedirs(f'{output_data}/test/', exist_ok=True)
    os.makedirs(f'{output_data}/validation/', exist_ok=True)
    file_root = str(int(time.time()*1000))
    
    # 데이터 세트 디스크에 저장
    print(f'데이터 세트 {output_data}에 저장하는 중...')
    tokenized_datasets['train'].to_parquet(f'./{output_data}/train/{file_root}.parquet')
    tokenized_datasets['test'].to_parquet(f'./{output_data}/test/{file_root}.parquet')
    tokenized_datasets['validation'].to_parquet(f'./{output_data}/validation/{file_root}.parquet')
    print('전처리 완료!')

In [ ]:
def process(args):

    print(f"내용 목록 - {args.input_data}")
    dirs_input = os.listdir(args.input_data)
    for file in dirs_input:
        print(file)

    transform_dataset(input_data=args.input_data, #'./data-summarization/',
                      output_data=args.output_data, #'./data-summarization-processed/',
                      huggingface_model_name=args.model_checkpoint, #model_checkpoint,
                      train_split_percentage=args.train_split_percentage, #0.90
                      test_split_percentage=args.test_split_percentage, #0.05
                      validation_split_percentage=args.validation_split_percentage, #0.05
                     )

    print(f"내용 목록 - {args.output_data}")
    dirs_output = os.listdir(args.output_data)
    for file in dirs_output:
        print(file)

# 데이터 세트 로컬에서 처리하기

In [ ]:
class Args:
    input_data: str
    output_data: str
    model_checkpoint: str
    train_split_percentage: float
    test_split_percentage: float
    validation_split_percentage: float

args = Args()

args.model_checkpoint = model_checkpoint
args.input_data = './data-summarization'
args.output_data = './data-summarization-processed'
args.train_split_percentage = 0.9
args.test_split_percentage = 0.05
args.validation_split_percentage = 0.05

# 이미 로컬에 저장된 데이터 제거
if os.path.isdir(args.output_data):
    import shutil
    shutil.rmtree(args.output_data)

process(args)

## 데이터 세트를 올바르게 불러왔는지 확인하기

In [ ]:
dataset = load_dataset(
    './data-summarization-processed/',
    data_files={'train': 'train/*.parquet', 'test': 'test/*.parquet', 'validation': 'validation/*.parquet'}
)
dataset